# Setting up a test subgraph and TRAPI endpoint for KG-Bioportal

## Prepare subgraph for ontologies relevant to clinical data

Load the set of ontologies to work with. This is defined in `clinical_ontologies.yaml`.

In [1]:
import yaml

In [9]:
test_ontologies = []
with open ('clinical_ontologies.yaml', 'r') as infile:
    ontologies_dict = yaml.safe_load(infile)

for ontology in ontologies_dict['ontologies']:
    try:
        if ontology['test_set']:
            test_ontologies.append(ontology['name'])
    except KeyError:
        continue

test_ontologies

['RADLEX', 'ATC']

Now build a graph of these alone. This assumes that the transformed Bioportal graphs are in `../transformed/ontologies/` by default, 

In [17]:
test_ontologies_str = ",".join(test_ontologies)

In [6]:
%cd ../
!python run.py catmerge --include_only {test_ontologies_str}

/home/harry
python: can't open file '/home/harry/run.py': [Errno 2] No such file or directory


## Set up and run Plater on its own

## Set up and run Automat